In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
PROJECT_DIR = "/content/drive/My Drive/Project_EnsembleLearning/"

In [3]:
import pandas as pd

In [4]:
# model output

df_prediction = pd.read_csv(PROJECT_DIR + 'predictions/prediction_neuralnet.csv')

df_prediction.head()

,Unnamed: 0,userId,movieId,rating,predicted_rating
0,0,1,307,3.5,1.874426
1,1,1,1590,2.5,1.791898
2,2,1,3424,4.5,3.000674
3,3,2,170,3.5,3.004210
4,4,2,1296,4.5,4.011952


In [5]:
# all movie ids vs titles

df_movies = \
  pd.read_csv(PROJECT_DIR + 'dataset_full/movies.csv', usecols=[0,1], names=['movieId', 'title'])

all_movies_dict = \
  pd.Series(df_movies['title'].values,index=df_movies['movieId']).to_dict()

In [6]:
# all users in the validation set 

all_users = df_prediction['userId'].unique().tolist()

In [7]:
# recommendations as per "predicted ratings" in the validation set 

df_prediction1 = df_prediction[['userId', 'movieId', 'predicted_rating']]
df_prediction1['predicted_movies'] = df_prediction1 \
  .apply(lambda x: (x['movieId'], x['predicted_rating']), axis=1)

df_prediction2 = df_prediction1[['userId', 'predicted_movies']]

df_prediction_formatted = df_prediction2 \
  .groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')

df_prediction_sorted = df_prediction_formatted

df_prediction_sorted['recommendation'] = \
  df_prediction_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

df_prediction_sorted.head()

sorted_reco_by_userid = pd.Series( \
  df_prediction_sorted['recommendation'].values,index=df_prediction_sorted['userId']).to_dict()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# this function returns top N recommendations for a given user
# based on the predicted ratings, N = 10 by default

def get_top_n_recommendations(user, N=10):
  top_n_reco = sorted_reco_by_userid[user][:N]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_reco]

In [9]:
# recommendations as per "true ratings" in the validation set

df_true1 = df_prediction[['userId', 'movieId', 'rating']]
df_true1['movies'] = df_true1 \
  .apply(lambda x: (x['movieId'], x['rating']), axis=1)

df_true2 = df_true1[['userId', 'movies']]

df_true_formatted = df_true2 \
  .groupby('userId')['movies'].apply(list).reset_index(name='recommendation')

df_true_sorted = df_true_formatted

df_true_sorted['recommendation'] = \
  df_true_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

df_true_sorted.head()

sorted_true_ratings_by_userid = pd.Series( \
  df_true_sorted['recommendation'].values,index=df_true_sorted['userId']).to_dict()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
# this function returns top N recommendations for a given user
# based on the true ratings, N = 10 by default

def get_top_n_true_ratings(user, N=10):
  top_n_true_ratings = sorted_true_ratings_by_userid[user][:N]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_true_ratings]

In [11]:
# get recommendation for each user in the validation set

top_n_recommendations_by_user = {}

user_out = []
for user in all_users:
  out = []

  # user
  out.append(user)

  # top N recommendations (n = 10 by default)
  top_n_reco_for_user = get_top_n_recommendations(user)
  top_n_recommendations_by_user[user] = top_n_reco_for_user

  top_n_true_ratings_for_user = get_top_n_true_ratings(user)
  true_positives = list(set(top_n_reco_for_user) & set(top_n_true_ratings_for_user))
  false_positives = list(set(top_n_reco_for_user) - set(top_n_true_ratings_for_user))
  false_negatives = list(set(top_n_true_ratings_for_user) - set(top_n_reco_for_user))

  # Compute precision for this user
  precision_for_user = len(true_positives) / float(len(true_positives) + len(false_positives))
  out.append(precision_for_user)

  # Compute recall for this user
  recall_for_user = len(true_positives) / float(len(true_positives) + len(false_negatives))
  out.append(recall_for_user)
  
  user_out.append(out)

In [12]:
# Recommendation evaluation for all users in the validation set

df_out = pd.DataFrame(user_out, columns=['userId', 'precision_user', 'recall_user'])
df_out

,userId,precision_user,recall_user
0,1,1.0,1.0
1,2,1.0,1.0
2,3,1.0,1.0
3,4,0.0,0.0
4,5,0.7,0.7
...,...,...,...
11408,11815,1.0,1.0
11409,11816,1.0,1.0
11410,11817,1.0,1.0
11411,11818,0.6,0.6


In [13]:
# top N recommendations for a user example

top_n_recommendations_by_user[7]

['About Schmidt (2002)',
 'When a Man Loves a Woman (1994)',
 "Rosemary's Baby (1968)"]

In [14]:
# EVALUATION

# mae
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(df_prediction['rating'], df_prediction['predicted_rating'])
print('mae', mae)

# rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(df_prediction['rating'], df_prediction['predicted_rating']))
print('rmse', rmse)

# precision
precision = df_out['precision_user'].mean()
print('avg precision', precision)

# recall
recall = df_out['recall_user'].mean()
print('avg recall', recall)

# f measure
f_measure = (2.0 * precision * recall) / (precision + recall)
print('f measure', f_measure)

mae 1.011494821352931
rmse 1.3070530429805927
avg precision 0.772995706650313
avg recall 0.772995706650313
f measure 0.772995706650313
